In [14]:
from statsmodels.tsa.api import AutoReg, VAR
from geopy.distance import great_circle
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r"/Users/main/Vault/Thesis/Data/pm25_weer.csv")
data

,id,no2,pm10,pm10_cal,pm10_fac,pm10_max,pm10_min,pm25,pm25_cal,pm25_fac,...,YYYYMMDD,jaar,maand,weeknummer,dag,H,T,U,DD,FH
0,1,NaN,8.195,8.774,1.070000,8.39,8.00,4.000,1.640,0.410000,...,20190712,2019,7,28,12,14,173,86,320,60
1,4,NaN,17.258,18.165,1.052000,20.10,15.60,11.176,4.256,0.381000,...,20190712,2019,7,28,12,14,173,86,320,60
2,5,NaN,19.952,20.497,1.027000,24.40,16.50,10.810,3.996,0.370000,...,20190712,2019,7,28,12,14,173,86,320,60
3,7,NaN,20.278,21.216,1.042000,24.40,16.20,11.510,4.292,0.371000,...,20190712,2019,7,28,12,14,173,86,320,60
4,10,NaN,18.099,19.186,1.062000,21.90,14.70,8.942,3.471,0.390000,...,20190712,2019,7,28,12,14,173,86,320,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349808,199,33.4319,24.660,16.796,0.681103,28.50,20.50,11.910,9.877,0.829303,...,20221022,2022,10,42,22,23,127,92,140,20
1349809,200,43.1974,0.000,0.000,NaN,0.00,0.00,0.000,0.000,NaN,...,20221022,2022,10,42,22,23,127,92,140,20
1349810,212,2.5000,23.074,22.407,0.971261,29.30,18.29,7.887,11.534,1.461850,...,20221022,2022,10,42,22,23,127,92,140,20
1349811,213,39.1156,37.694,36.569,0.970257,60.80,24.40,9.182,13.432,1.463180,...,20221022,2022,10,42,22,23,127,92,140,20


In [54]:
data.columns

Index(['id', 'no2', 'pm10', 'pm10_cal', 'pm10_fac', 'pm10_max', 'pm10_min',
       'pm25', 'pm25_cal', 'pm25_fac', 'pm25_max', 'pm25_min', 'timestamp',
       'components', 'latitude', 'longitude', 'name', 'sensortype', 'tag',
       'datum', 'tijd', 'weekdag', 'uur', '#STN', 'YYYYMMDD', 'jaar', 'maand',
       'weeknummer', 'dag', 'H', 'T', 'U', 'DD', 'FH'],
      dtype='object')

In [3]:
data.drop(data.iloc[:, 0:7], axis=1, inplace=True)
data.drop(["jaar", "maand", "weeknummer", "#STN", "timestamp", "components", "dag", "tijd", "datum", "H", "T", "U", "DD", "weekdag", "FH", "sensortype"], axis=1, inplace=True)
data

,pm25,pm25_cal,pm25_fac,pm25_max,pm25_min,latitude,longitude,name,tag,uur,YYYYMMDD
0,4.000,1.640,0.410000,4.1,3.9,52.472796,4.662902,Zuiderkade,Beverwijk,14,20190712
1,11.176,4.256,0.381000,12.4,10.1,52.490108,4.657723,Grote Houtweg,Beverwijk,14,20190712
2,10.810,3.996,0.370000,12.3,9.0,52.484320,4.642918,Westerhoutplein,Beverwijk,14,20190712
3,11.510,4.292,0.371000,13.9,10.0,52.487020,4.650807,Beukenstraat,Beverwijk,14,20190712
4,8.942,3.471,0.390000,10.3,7.3,52.477398,4.658354,Parallelweg,Beverwijk,14,20190712
...,...,...,...,...,...,...,...,...,...,...,...
1349808,11.910,9.877,0.829303,15.2,10.3,52.473001,4.662653,Zuiderkade,Beverwijk,23,20221022
1349809,0.000,0.000,NaN,0.0,0.0,52.477803,4.654015,Stationsplein,Beverwijk,23,20221022
1349810,7.887,11.534,1.461850,8.7,7.2,52.458385,4.834105,Fluitekruidweg,Zaandam,23,20221022
1349811,9.182,13.432,1.463180,10.5,8.3,52.458385,4.834105,Fluitekruidweg,Zaandam,23,20221022


In [ ]:
data = data[data.pm25 != 0]
data

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(data["pm25"])
plt.show()

In [ ]:
data3 = data["pm25"].to_numpy()

plt.figure(figsize=(16, 8))
plt.plot(data3[700000:700500])
plt.show()

In [4]:
grouped_df = data.groupby(["YYYYMMDD", "tag"])[("pm25", "longitude", "latitude")].mean().copy().reset_index()

/var/folders/7f/x1mp2wjd10vcy_krqb5_kwjc0000gn/T/ipykernel_14816/3703936478.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_df = data.groupby(["YYYYMMDD", "tag"])[("pm25", "longitude", "latitude")].mean().copy().reset_index()


In [5]:
grouped_df.head(5)

,YYYYMMDD,tag,pm25,longitude,latitude
0,20190625,Amsterdam,16.602565,4.866208,52.359714
1,20190625,Beverwijk,18.822000,4.650824,52.492369
2,20190626,Amsterdam,21.974609,4.866208,52.359714
3,20190626,Beverwijk,2.851417,4.657933,52.485871
4,20190627,Amsterdam,4.484500,4.866208,52.359714


In [6]:
Locations = grouped_df["tag"].unique()
LocDict = dict()
Locations
for i in range(len(Locations)):
    LocDict[Locations[i]] = (grouped_df[grouped_df.tag == Locations[i]]["latitude"].mean(), grouped_df[grouped_df.tag == Locations[i]]["longitude"].mean())

In [7]:
LocDict

{'Amsterdam': (52.385565471780744, 4.901327224364789),
 'Beverwijk': (52.486366756022534, 4.658584737682753),
 'Heemskerk': (52.50593616684639, 4.671953092146708),
 'Wijk aan Zee': (52.493443775747366, 4.59824578234632),
 'Velsen-Noord': (52.47382287197285, 4.647238399148998),
 'Driehuis': (52.447194827537714, 4.636977030738782),
 'IJmuiden': (52.45820717461739, 4.615567828011168),
 'Velsen-Zuid': (52.46234499999992, 4.637255999999981),
 'Zaandam': (52.45848175659801, 4.824912486862412),
 'Koog aan de Zaan': (52.46407900000035, 4.811263999999972),
 'Uithoorn': (52.23809099999961, 4.8082289999999865)}

In [36]:
W = np.zeros((11, 11))

for i in range(len(LocDict)):
    for j in range(len(LocDict)):
        if i != j:
            W[i, j] = 1 / great_circle(LocDict[Locations[i]], LocDict[Locations[j]]).km
pd.DataFrame(W)


,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,0.050226,0.048748,0.042035,0.050438,0.052102,0.047633,0.050410,0.103925,0.093859,0.056891
1,0.050226,0.000000,0.424320,0.240358,0.627957,0.217623,0.233800,0.329302,0.085581,0.094041,0.033993
2,0.048748,0.424320,0.000000,0.193082,0.253583,0.143926,0.152950,0.185648,0.086023,0.095066,0.032065
3,0.042035,0.240358,0.193082,0.000000,0.251844,0.173214,0.244498,0.229791,0.063146,0.067607,0.031474
4,0.050438,0.627957,0.253583,0.251844,0.000000,0.328792,0.362306,0.692351,0.082260,0.089577,0.035210
5,0.052102,0.217623,0.143926,0.173214,0.328792,0.000000,0.526740,0.593567,0.078144,0.083627,0.038463
6,0.047633,0.233800,0.152950,0.244498,0.362306,0.526740,0.000000,0.649445,0.070501,0.075331,0.036030
7,0.050410,0.329302,0.185648,0.229791,0.692351,0.593567,0.649445,0.000000,0.078608,0.084816,0.036354
8,0.103925,0.085581,0.086023,0.063146,0.082260,0.078144,0.070501,0.078608,0.000000,0.897148,0.040762
9,0.093859,0.094041,0.095066,0.067607,0.089577,0.083627,0.075331,0.084816,0.897148,0.000000,0.039794


In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(grouped_df["pm25"])
plt.show()

In [20]:
data2 = grouped_df.copy()
data2["New"] = data2["YYYYMMDD"].astype(str) #+ "-" +data2["uur"].astype(str)
del data2["YYYYMMDD"]

In [21]:
data2.set_index("New", inplace=True)

In [25]:
del data2["latitude"]
del data2["longitude"]
data2

,tag,pm25
New,,
20190625,Amsterdam,16.602565
20190625,Beverwijk,18.822000
20190626,Amsterdam,21.974609
20190626,Beverwijk,2.851417
20190627,Amsterdam,4.484500
...,...,...
20221022,IJmuiden,6.112711
20221022,Uithoorn,7.120000
20221022,Velsen-Noord,6.428708


In [26]:
UniqueNames = data2.tag.unique()

DataFrameDict = {elem : pd.DataFrame() for elem in UniqueNames}

for key in DataFrameDict.keys():
    DataFrameDict[key] = data2[:][data2.tag == key]

In [27]:
DataFrameDict["Amsterdam"]

,tag,pm25
New,,
20190625,Amsterdam,16.602565
20190626,Amsterdam,21.974609
20190627,Amsterdam,4.484500
20190628,Amsterdam,7.430917
20190629,Amsterdam,7.474792
...,...,...
20221018,Amsterdam,59.648775
20221019,Amsterdam,49.008481
20221020,Amsterdam,57.314282


In [28]:
for key in tqdm(DataFrameDict):
    print(key)
    plt.figure(figsize=(16, 8))
    plt.plot(DataFrameDict[key].pm25, label = key)
    plt.show()

  0%|          | 0/11 [00:00<?, ?it/s]

Amsterdam


  0%|          | 0/11 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [29]:
for key in tqdm(DataFrameDict):
    DataFrameDict[key].rename(columns={"pm25":key}, inplace=True)
    del DataFrameDict[key]["tag"]
    print(DataFrameDict[key])

100%|██████████| 11/11 [00:00<00:00, 406.27it/s]

          Amsterdam
New                
20190625  16.602565
20190626  21.974609
20190627   4.484500
20190628   7.430917
20190629   7.474792
...             ...
20221018  59.648775
20221019  49.008481
20221020  57.314282
20221021  68.504173
20221022  59.205616

[1195 rows x 1 columns]
          Beverwijk
New                
20190625  18.822000
20190626   2.851417
20190627   4.010603
20190628   5.533953
20190629   9.006055
...             ...
20221018  44.425756
20221019  45.131791
20221020  55.310167
20221021  56.444523
20221022  44.434752

[1204 rows x 1 columns]
          Heemskerk
New                
20190702   3.702000
20190703   2.701783
20190704   2.456625
20190705   4.618917
20190706   6.488875
...             ...
20221018   3.995484
20221019   4.602050
20221020  15.024389
20221021   8.620153
20221022   4.203480

[1186 rows x 1 columns]
          Wijk aan Zee
New                   
20190703      2.059048
20190704      1.748930
20190705      7.951859
20190706      6.433143
2019070

In [ ]:
for key in tqdm(DataFrameDict):
    model = AutoReg(DataFrameDict[key], lags=1).fit()
    print(f'{key}\n', model.params)

In [30]:
for key in DataFrameDict:
    print(key)

Amsterdam
Beverwijk
Heemskerk
Wijk aan Zee
Velsen-Noord
Driehuis
IJmuiden
Velsen-Zuid
Zaandam
Koog aan de Zaan
Uithoorn


In [31]:
for key in tqdm(DataFrameDict):
    for key2 in tqdm(DataFrameDict):
        idx1 = DataFrameDict[key].index
        idx2 = DataFrameDict[key2].index
        print(len(idx1.difference(idx2)))

100%|██████████| 11/11 [00:00<00:00, 867.59it/s]


0
0
19
13
117
75
21
577
151
670
816


100%|██████████| 11/11 [00:00<00:00, 1236.66it/s]


9
0
19
19
123
84
26
586
154
679
825


100%|██████████| 11/11 [00:00<00:00, 931.58it/s]


...

7
1
4
12
53
62
9
479
0
526
672


100%|██████████| 11/11 [00:00<00:00, 1400.73it/s]


0
0
0
0
17
15
0
194
0
0
265


100%|██████████| 11/11 [00:00<00:00, 62.97it/s]

0
0
0
0
0
0
0
229
0
119
0


In [32]:
df = pd.DataFrame(DataFrameDict["Amsterdam"].copy())
for key in DataFrameDict:
    df = df.combine_first(DataFrameDict[key])
df

,Amsterdam,Beverwijk,Driehuis,Heemskerk,IJmuiden,Koog aan de Zaan,Uithoorn,Velsen-Noord,Velsen-Zuid,Wijk aan Zee,Zaandam
New,,,,,,,,,,,
20190625,16.602565,18.822000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190626,21.974609,2.851417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190627,4.484500,4.010603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190628,7.430917,5.533953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190629,7.474792,9.006055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20221018,59.648775,44.425756,7.785098,3.995484,4.095699,NaN,5.980941,5.637824,NaN,117.669196,7.467459
20221019,49.008481,45.131791,11.062278,4.602050,5.595226,NaN,5.646750,6.915667,NaN,117.688819,5.922350
20221020,57.314282,55.310167,36.090788,15.024389,22.693654,NaN,27.862864,25.996136,NaN,130.220561,23.761873


In [33]:
for column in df:
    mean_value = df[column].mean()
    df[column].fillna(value=mean_value, inplace = True)
df

,Amsterdam,Beverwijk,Driehuis,Heemskerk,IJmuiden,Koog aan de Zaan,Uithoorn,Velsen-Noord,Velsen-Zuid,Wijk aan Zee,Zaandam
New,,,,,,,,,,,
20190625,16.602565,18.822000,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
20190626,21.974609,2.851417,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
20190627,4.484500,4.010603,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
20190628,7.430917,5.533953,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
20190629,7.474792,9.006055,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
...,...,...,...,...,...,...,...,...,...,...,...
20221018,59.648775,44.425756,7.785098,3.995484,4.095699,15.217022,5.980941,5.637824,9.501286,117.669196,7.467459
20221019,49.008481,45.131791,11.062278,4.602050,5.595226,15.217022,5.646750,6.915667,9.501286,117.688819,5.922350
20221020,57.314282,55.310167,36.090788,15.024389,22.693654,15.217022,27.862864,25.996136,9.501286,130.220561,23.761873


In [34]:
VARModel = VAR(df).fit()
VARModel.summary()

/Users/main/opt/miniconda3/envs/RA/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 14, Dec, 2022
Time:                     22:58:57
--------------------------------------------------------------------
No. of Equations:         11.0000    BIC:                    39.3860
Nobs:                     1204.00    HQIC:                   39.0379
Log likelihood:          -42034.6    FPE:                7.28826e+16
AIC:                      38.8276    Det(Omega_mle):     6.53501e+16
--------------------------------------------------------------------
Results for equation Amsterdam
                         coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------------
const                       6.387706         1.088960            5.866           0.000
L1.Amsterdam                0.710798         0.037650           18.879           0.000
L1.Beverwijk                0.04